In [13]:
import sys
print('python', sys.version)

import os

import numpy as np
print('numpy', np.__version__)

import pandas as pd
print('pandas', pd.__version__)

import matplotlib as mpl
print('matplotlib', mpl.__version__)

import matplotlib.pyplot as plt
import scipy.stats
import seaborn as sns

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

python 3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]
numpy 1.19.1
pandas 1.1.1
matplotlib 3.3.1


## mapping human gene symbol to entrez

In [3]:
# mapping human gene symbol to entrez
dat=pd.read_table('data_ori/Sym2Entrez.txt',sep='\t')
dat.columns=['sym','entrez']
dat=dat.loc[dat['entrez'].notnull()]
dat=dat.loc[dat['sym'].notnull()]
dat['entrez']=dat['entrez'].astype(int).astype(str)

e2s=dat.set_index('entrez')
e2s_dic=e2s['sym'].to_dict() # e2s_dic={'1': 'A1BG', '503538': 'A1BG-AS1': ,...}

## GI prediction (with CRISPER)

### read expression file

In [14]:
exp_ori=pd.read_table('data_ori/CCLE_expression.csv',index_col=0, sep=',') # CCLE_expression, 19144 genes, 1305 cells
print('# of nan: ', exp_ori.isnull().sum().sum()) # not containing nan value

exp_ori.columns=[gene.split(' ')[1] for gene in exp_ori.columns]
exp_ori.columns=[col[1:-1] for col in exp_ori.columns]
exp_ori.columns=[e2s_dic[col] if col in e2s_dic.keys() else 'NA_symbol' for col in exp_ori.columns]
exp_ori=exp_ori.loc[:,exp_ori.columns!='NA_symbol']

exp_arr=np.array(exp_ori)
print('# of ori expresions: ', exp_arr.flatten().shape)

exp_nonzero=exp_arr[exp_arr!=0]
print('# of non-zero expresions: ', exp_nonzero.shape)

exp_ori.index.name='DepMap_ID'
exp_ori.head(3)
exp_ori.shape

# of nan:  0
# of ori expresions:  (24975090,)
# of non-zero expresions:  (20781966,)


,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,SCYGR3,SCYGR7,OR8S1,SCYGR6,OR4F29,EEF1AKMT4,TBCE,OOSP4A,ARHGAP11B,OOSP3
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-001097,0.000000,0.0,4.667324,1.761285,3.554589,4.358959,0.641546,2.201634,1.941106,2.150560,...,0.0,0.0,0.0,0.0,0.0,2.980025,5.132988,0.0,0.443607,0.000000
ACH-001636,0.000000,0.0,6.198887,2.032101,3.755956,2.601697,0.000000,0.000000,3.415488,3.620586,...,0.0,0.0,0.0,0.0,0.0,4.322649,4.100978,0.0,2.589763,0.298658
ACH-001804,4.934988,0.0,6.635464,1.929791,2.503349,0.000000,4.883621,6.772546,3.889474,4.139961,...,0.0,0.0,0.0,0.0,0.0,2.283922,4.288359,0.0,0.475085,0.000000


(1305, 19138)

### read mutation profile

In [4]:
mut_cnt=pd.read_table('data_preproc/mut_varL.txt',index_col=0, sep='\t')

mut_cnt.columns=[e2s_dic[col] if col in e2s_dic.keys() else 'NA_symbol' for col in mut_cnt.columns]
mut_cnt=mut_cnt.loc[:,mut_cnt.columns!='NA_symbol']

mut_cnt.head(3)

,A1BG,A2M,NAT1,NAT2,SERPINA3,AADAC,AAMP,AANAT,AARS1,ABAT,...,PTGES3L,DNAH17-AS1,FMC1-LUC7L2,PYURF,OVCH1-AS1,GOLGA8R,NPIPA8,ZNF891,SPECC1L-ADORA2A,MEI4
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACH-000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACH-000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### read depletion profile

In [5]:
dep_crp=pd.read_table('data_preproc/dep_crisper.txt',index_col=0, sep='\t')

dep_crp.columns=[col[1:-1] for col in dep_crp.columns]
dep_crp.columns=[e2s_dic[col] if col in e2s_dic.keys() else 'NA_symbol' for col in dep_crp.columns]
dep_crp=dep_crp.loc[:,dep_crp.columns!='NA_symbol']

dep_crp.head(3)

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZSWIM8,ZUP1,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000004,0.178087,0.088290,-0.201464,-0.017011,0.042590,-0.192257,0.352891,-0.445209,0.292603,0.165168,...,-0.174114,-0.161678,-0.160386,-0.122795,-0.464567,0.258084,0.237593,-0.408563,0.291832,0.218575
ACH-000005,-0.082364,0.239413,0.195457,0.163763,-0.188161,-0.338735,0.254489,-0.583878,-0.060615,0.015272,...,-0.273881,-0.231005,-0.497114,-0.195061,-0.408063,0.229242,-0.075883,-0.104001,0.075901,0.036985
ACH-000007,0.070026,0.075066,-0.064637,0.156905,0.099075,0.141985,0.066314,-0.477023,-0.006855,0.291604,...,-0.131611,-0.106505,-0.580785,-0.093252,-0.276874,0.211699,-0.012654,-0.351716,0.089140,-0.392549


### get shared cells between mut_count and dep_crisper

In [6]:
shared_index=set(mut_cnt.index) & set(dep_crp.index) & set(exp_ori.index)
shared_columns=set(dep_crp.columns) & set(exp_ori.columns)
mut_sh=mut_cnt.loc[shared_index]
dep_sh=dep_crp.loc[shared_index, shared_columns]
exp_sh=exp_ori.loc[shared_index, shared_columns]

mut_sh.head(3)
mut_sh.shape
dep_sh.head(3)
dep_sh.shape
exp_sh.head(3)
exp_sh.shape

,A1BG,A2M,NAT1,NAT2,SERPINA3,AADAC,AAMP,AANAT,AARS1,ABAT,...,PTGES3L,DNAH17-AS1,FMC1-LUC7L2,PYURF,OVCH1-AS1,GOLGA8R,NPIPA8,ZNF891,SPECC1L-ADORA2A,MEI4
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACH-000846,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACH-000390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


(763, 18725)

,MTERF4,P2RY12,CLIC6,ILDR2,CTU1,GABRP,CCNJL,FAM20A,NKX3-2,C1orf174,...,ASB5,CERS6,WNT8A,BARD1,VPS13C,GOPC,PIK3IP1,ZNF304,EIF5A2,ZNF606
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000128,-0.526837,-0.189314,0.094133,-0.150847,-0.182924,0.069279,-0.106692,-0.065677,0.161112,0.078415,...,0.097745,0.053205,0.061174,-0.542859,0.003359,0.022210,-0.091866,-0.146314,0.097409,-0.126725
ACH-000846,-0.238960,-0.286652,-0.308171,-0.328692,-0.177624,-0.024497,0.025977,-0.075048,0.395494,0.159638,...,0.045691,0.133582,-0.025413,-0.667395,0.095281,0.026860,-0.053670,0.160926,0.048642,-0.343963
ACH-000390,-0.289327,-0.161952,-0.093481,-0.178759,-0.553631,0.135614,-0.116953,-0.180268,0.252873,0.156711,...,0.259426,0.092418,0.022015,-0.810391,0.083888,0.103754,-0.010503,0.120061,0.011264,-0.106893


(763, 16105)

,MTERF4,P2RY12,CLIC6,ILDR2,CTU1,GABRP,CCNJL,FAM20A,NKX3-2,C1orf174,...,ASB5,CERS6,WNT8A,BARD1,VPS13C,GOPC,PIK3IP1,ZNF304,EIF5A2,ZNF606
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000128,5.118111,0.056584,3.209453,4.048759,1.448901,0.028569,2.375735,1.144046,1.521051,3.781360,...,0.014355,4.428946,0.0,4.233428,3.718088,4.185074,1.304511,2.475085,3.073820,3.463361
ACH-000846,5.320485,0.084064,0.014355,0.111031,1.956057,0.555816,3.871844,0.356144,0.070389,4.193772,...,0.000000,4.028569,0.0,3.346248,4.012569,4.615299,1.176323,2.867896,3.218781,3.509696
ACH-000390,4.828327,0.000000,0.028569,0.056584,2.087463,0.000000,2.627607,1.097611,0.000000,3.902074,...,0.000000,2.490570,0.0,2.757023,3.294253,3.427606,1.111031,2.395063,2.922198,3.077243


(763, 16105)

### GI score by ttest

In [7]:
#_=np.seterr('raise')

In [8]:
def perform_ttest(mut_sel, dep_sel, exp_sel):
    
    meta_info='CRISPER_M{}_I{}'.format(str(MIN_MUT_PER).replace('.',''), MIN_MUT_NUM)
    
    # open file for SL score
    SL_WO_Exp=open('result_SL_val/{}_WO_Exp.txt'.format(meta_info),'w')
    SL_WI_Exp=open('result_SL_val/{}_WI_Exp.txt'.format(meta_info),'w')
    _=SL_WO_Exp.write('\t'.join(['mut_gene','dep_gene','t_val','p_val'])+'\n')
    _=SL_WI_Exp.write('\t'.join(['mut_gene','dep_gene','t_val','p_val'])+'\n')

    # select genes mutated frequently
    mut_num=np.sum(mut_sel!=0, axis=0)
    mut_per=np.sum(mut_sel!=0, axis=0)/len(mut_sel)
    mut_sel=mut_sel.loc[:,(mut_num>=MIN_MUT_NUM)&(mut_per>=MIN_MUT_PER)]
    
    mut_g_cnt=0
    for mut_g in mut_sel.columns:
        mut_g_cnt+=1
        print('processing : {}/{}'.format(mut_g_cnt, len(mut_sel.columns)))
        mut_sr=mut_sel[mut_g]
        
        for dep_g in list(dep_sel):
            dep_sr=dep_sel[dep_g]
            exp_sr=exp_sel[dep_g]
            
            # merge mutation, depletion, expression
            mer_df=pd.concat([mut_sr,dep_sr,exp_sr], axis=1, join='inner')
            mer_df.columns=['mut','dep','exp']

            # ttest without considering expression data
            con_dep_score=mer_df.loc[mer_df['mut']==0, 'dep']
            mut_dep_score=mer_df.loc[mer_df['mut']!=0, 'dep']
                        
            t_val, p_val = scipy.stats.ttest_ind(con_dep_score, mut_dep_score, equal_var=False)
            _=SL_WO_Exp.write('{}\t{}\t{:.5f}\t{:.5e}\n'.format(mut_g, dep_g, t_val, p_val))
            
            # ttest with considering expression data
            # obtain instance with nonzero expression
            mer_df=mer_df.loc[mer_df['exp']!=0]
            con_dep_score=mer_df.loc[mer_df['mut']==0, 'dep']
            mut_dep_score=mer_df.loc[mer_df['mut']!=0, 'dep']
            
            if (len(mut_dep_score) < MIN_MUT_NUM) or (len(mut_dep_score)/len(mer_df) < MIN_MUT_PER):
                continue
                
            t_val, p_val = scipy.stats.ttest_ind(con_dep_score, mut_dep_score, equal_var=False)
            _=SL_WI_Exp.write('{}\t{}\t{:.5f}\t{:.5e}\n'.format(mut_g, dep_g, t_val, p_val))
        
    SL_WO_Exp.close()
    SL_WI_Exp.close()

In [9]:
MIN_MUT_PER=0.03 ## minimum mutation occurance percentage
MIN_MUT_NUM=10

perform_ttest(mut_sh, dep_sh, exp_sh) # t_val: positive --> SL signature    

processing : 1/5030
processing : 2/5030
processing : 3/5030
processing : 4/5030
processing : 5/5030
processing : 6/5030
processing : 7/5030
processing : 8/5030
processing : 9/5030
processing : 10/5030
processing : 11/5030
processing : 12/5030
processing : 13/5030
processing : 14/5030
processing : 15/5030
processing : 16/5030
processing : 17/5030
processing : 18/5030
processing : 19/5030
processing : 20/5030
processing : 21/5030
processing : 22/5030
processing : 23/5030
processing : 24/5030
processing : 25/5030
processing : 26/5030
processing : 27/5030
processing : 28/5030
processing : 29/5030
processing : 30/5030
processing : 31/5030
processing : 32/5030
processing : 33/5030
processing : 34/5030
processing : 35/5030
processing : 36/5030
processing : 37/5030
processing : 38/5030
processing : 39/5030
processing : 40/5030
processing : 41/5030
processing : 42/5030
processing : 43/5030
processing : 44/5030
processing : 45/5030
processing : 46/5030
processing : 47/5030
processing : 48/5030
p

processing : 379/5030
processing : 380/5030
processing : 381/5030
processing : 382/5030
processing : 383/5030
processing : 384/5030
processing : 385/5030
processing : 386/5030
processing : 387/5030
processing : 388/5030
processing : 389/5030
processing : 390/5030
processing : 391/5030
processing : 392/5030
processing : 393/5030
processing : 394/5030
processing : 395/5030
processing : 396/5030
processing : 397/5030
processing : 398/5030
processing : 399/5030
processing : 400/5030
processing : 401/5030
processing : 402/5030
processing : 403/5030
processing : 404/5030
processing : 405/5030
processing : 406/5030
processing : 407/5030
processing : 408/5030
processing : 409/5030
processing : 410/5030
processing : 411/5030
processing : 412/5030
processing : 413/5030
processing : 414/5030
processing : 415/5030
processing : 416/5030
processing : 417/5030
processing : 418/5030
processing : 419/5030
processing : 420/5030
processing : 421/5030
processing : 422/5030
processing : 423/5030
processing

processing : 752/5030
processing : 753/5030
processing : 754/5030
processing : 755/5030
processing : 756/5030
processing : 757/5030
processing : 758/5030
processing : 759/5030
processing : 760/5030
processing : 761/5030
processing : 762/5030
processing : 763/5030
processing : 764/5030
processing : 765/5030
processing : 766/5030
processing : 767/5030
processing : 768/5030
processing : 769/5030
processing : 770/5030
processing : 771/5030
processing : 772/5030
processing : 773/5030
processing : 774/5030
processing : 775/5030
processing : 776/5030
processing : 777/5030
processing : 778/5030
processing : 779/5030
processing : 780/5030
processing : 781/5030
processing : 782/5030
processing : 783/5030
processing : 784/5030
processing : 785/5030
processing : 786/5030
processing : 787/5030
processing : 788/5030
processing : 789/5030
processing : 790/5030
processing : 791/5030
processing : 792/5030
processing : 793/5030
processing : 794/5030
processing : 795/5030
processing : 796/5030
processing

processing : 1119/5030
processing : 1120/5030
processing : 1121/5030
processing : 1122/5030
processing : 1123/5030
processing : 1124/5030
processing : 1125/5030
processing : 1126/5030
processing : 1127/5030
processing : 1128/5030
processing : 1129/5030
processing : 1130/5030
processing : 1131/5030
processing : 1132/5030
processing : 1133/5030
processing : 1134/5030
processing : 1135/5030
processing : 1136/5030
processing : 1137/5030
processing : 1138/5030
processing : 1139/5030
processing : 1140/5030
processing : 1141/5030
processing : 1142/5030
processing : 1143/5030
processing : 1144/5030
processing : 1145/5030
processing : 1146/5030
processing : 1147/5030
processing : 1148/5030
processing : 1149/5030
processing : 1150/5030
processing : 1151/5030
processing : 1152/5030
processing : 1153/5030
processing : 1154/5030
processing : 1155/5030
processing : 1156/5030
processing : 1157/5030
processing : 1158/5030
processing : 1159/5030
processing : 1160/5030
processing : 1161/5030
processing 

C:\Users\jmjun\Anaconda3\envs\SL\lib\site-packages\numpy\core\fromnumeric.py:3621: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\jmjun\Anaconda3\envs\SL\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\jmjun\Anaconda3\envs\SL\lib\site-packages\numpy\core\fromnumeric.py:3621: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\jmjun\Anaconda3\envs\SL\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\jmjun\Anaconda3\envs\SL\lib\site-packages\numpy\core\fromnumeric.py:3621: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\jmjun\Anaconda3\envs\SL\lib\si

...

processing : 3531/5030
processing : 3532/5030
processing : 3533/5030
processing : 3534/5030
processing : 3535/5030
processing : 3536/5030
processing : 3537/5030
processing : 3538/5030
processing : 3539/5030
processing : 3540/5030
processing : 3541/5030
processing : 3542/5030
processing : 3543/5030
processing : 3544/5030
processing : 3545/5030
processing : 3546/5030
processing : 3547/5030
processing : 3548/5030
processing : 3549/5030
processing : 3550/5030
processing : 3551/5030
processing : 3552/5030
processing : 3553/5030
processing : 3554/5030
processing : 3555/5030
processing : 3556/5030
processing : 3557/5030
processing : 3558/5030
processing : 3559/5030
processing : 3560/5030
processing : 3561/5030
processing : 3562/5030
processing : 3563/5030
processing : 3564/5030
processing : 3565/5030
processing : 3566/5030
processing : 3567/5030
processing : 3568/5030
processing : 3569/5030
processing : 3570/5030
processing : 3571/5030
processing : 3572/5030
processing : 3573/5030
processing 

processing : 3888/5030
processing : 3889/5030
processing : 3890/5030
processing : 3891/5030
processing : 3892/5030
processing : 3893/5030
processing : 3894/5030
processing : 3895/5030
processing : 3896/5030
processing : 3897/5030
processing : 3898/5030
processing : 3899/5030
processing : 3900/5030
processing : 3901/5030
processing : 3902/5030
processing : 3903/5030
processing : 3904/5030
processing : 3905/5030
processing : 3906/5030
processing : 3907/5030
processing : 3908/5030
processing : 3909/5030
processing : 3910/5030
processing : 3911/5030
processing : 3912/5030
processing : 3913/5030
processing : 3914/5030
processing : 3915/5030
processing : 3916/5030
processing : 3917/5030
processing : 3918/5030
processing : 3919/5030
processing : 3920/5030
processing : 3921/5030
processing : 3922/5030
processing : 3923/5030
processing : 3924/5030
processing : 3925/5030
processing : 3926/5030
processing : 3927/5030
processing : 3928/5030
processing : 3929/5030
processing : 3930/5030
processing 

processing : 4245/5030
processing : 4246/5030
processing : 4247/5030
processing : 4248/5030
processing : 4249/5030
processing : 4250/5030
processing : 4251/5030
processing : 4252/5030
processing : 4253/5030
processing : 4254/5030
processing : 4255/5030
processing : 4256/5030
processing : 4257/5030
processing : 4258/5030
processing : 4259/5030
processing : 4260/5030
processing : 4261/5030
processing : 4262/5030
processing : 4263/5030
processing : 4264/5030
processing : 4265/5030
processing : 4266/5030
processing : 4267/5030
processing : 4268/5030
processing : 4269/5030
processing : 4270/5030
processing : 4271/5030
processing : 4272/5030
processing : 4273/5030
processing : 4274/5030
processing : 4275/5030
processing : 4276/5030
processing : 4277/5030
processing : 4278/5030
processing : 4279/5030
processing : 4280/5030
processing : 4281/5030
processing : 4282/5030
processing : 4283/5030
processing : 4284/5030
processing : 4285/5030
processing : 4286/5030
processing : 4287/5030
processing 

processing : 4602/5030
processing : 4603/5030
processing : 4604/5030
processing : 4605/5030
processing : 4606/5030
processing : 4607/5030
processing : 4608/5030
processing : 4609/5030
processing : 4610/5030
processing : 4611/5030
processing : 4612/5030
processing : 4613/5030
processing : 4614/5030
processing : 4615/5030
processing : 4616/5030
processing : 4617/5030
processing : 4618/5030
processing : 4619/5030
processing : 4620/5030
processing : 4621/5030
processing : 4622/5030
processing : 4623/5030
processing : 4624/5030
processing : 4625/5030
processing : 4626/5030
processing : 4627/5030
processing : 4628/5030
processing : 4629/5030
processing : 4630/5030
processing : 4631/5030
processing : 4632/5030
processing : 4633/5030
processing : 4634/5030
processing : 4635/5030
processing : 4636/5030
processing : 4637/5030
processing : 4638/5030
processing : 4639/5030
processing : 4640/5030
processing : 4641/5030
processing : 4642/5030
processing : 4643/5030
processing : 4644/5030
processing 

processing : 4959/5030
processing : 4960/5030
processing : 4961/5030
processing : 4962/5030
processing : 4963/5030
processing : 4964/5030
processing : 4965/5030
processing : 4966/5030
processing : 4967/5030
processing : 4968/5030
processing : 4969/5030
processing : 4970/5030
processing : 4971/5030
processing : 4972/5030
processing : 4973/5030
processing : 4974/5030
processing : 4975/5030
processing : 4976/5030
processing : 4977/5030
processing : 4978/5030
processing : 4979/5030
processing : 4980/5030
processing : 4981/5030
processing : 4982/5030
processing : 4983/5030
processing : 4984/5030
processing : 4985/5030
processing : 4986/5030
processing : 4987/5030
processing : 4988/5030
processing : 4989/5030
processing : 4990/5030
processing : 4991/5030
processing : 4992/5030
processing : 4993/5030
processing : 4994/5030
processing : 4995/5030
processing : 4996/5030
processing : 4997/5030
processing : 4998/5030
processing : 4999/5030
processing : 5000/5030
processing : 5001/5030
processing 

In [10]:
# import statsmodels.stats.multitest as mt

# res=pd.read_table('result_SL/CRISPER_M003_I10_SL.txt',sep='\t')
# res
# res['fdr']=mt.multipletests(res['p_val'],method='fdr_bh')[1]
# res
# res.to_csv('result_SL/CRISPER_M003_I10_SL_fdr.txt',index=False,sep='\t')
# pd.read_table('result_SL/CRISPER_M003_I10_SL_fdr.txt',sep='\t')